# Database Loading Tool
This notebook lets you insert data sets into the local Supabase db from the source PKL files.

In [11]:
import pandas as pd

# Load the PKL files for all sources
food_network = pd.read_pickle('cleaned_food_network_restaurants_with_validated_addresses.pkl.zst', compression='zstd')



In [12]:
# Update typo in show name

food_network.loc[food_network['show'] == 'Diners, Dive-Ins, and Dives', 'show'] = 'Diners, Drive-Ins, and Dives'

# Normalization
We must take the input files and destructure them into normalized entities so that we may populate our DB

In [13]:
# Extract Shows

food_network_show_names = food_network['show'].unique()
# Make them into their own dataframe
shows = pd.DataFrame(food_network_show_names, columns=['name'])
# Add the platform  and publisher names as a columns
shows['publisher'] = 'Food Network'
shows['platform'] = 'Television'
shows.drop_duplicates(subset=['name'], inplace=True)

# Dedupe
food_network.drop_duplicates(subset=['show', 'restaurant', 'validated_address'], inplace=True)

# Drop unnecessary columns
food_network = food_network[['show', 'restaurant', 'description', 'rating', 'phone',  'website', 'website status', 'validated_address', 'city', 'state', 'zip']]
# Rename columns
food_network.columns = ['show', 'name', 'description', 'rating', 'phone', 'website', 'website_status', 'address', 'city', 'state', 'zip']

# Extract Restaurants
food_network_restaurants = food_network[['name', 'description', 'rating', 'phone',  'website', 'website_status', 'address', 'city', 'state', 'zip']]
# Dedupe restaurants that may have been on multiple shows
food_network_restaurants = food_network_restaurants.drop_duplicates(subset=['name', 'address'])


In [14]:
from sqlalchemy import create_engine, text
import config

# Prod Variant:
engine = create_engine(f'postgresql://{config.PROD_DB_USER}:{config.PROD_DB_PASSWORD}@{config.PROD_DB_HOST}:{config.PROD_DB_PORT}/{config.PROD_DB_NAME}?gssencmode=disable')

# Local Variant:
# engine = create_engine(f'postgresql://{config.LOCAL_DB_USER}:{config.LOCAL_DB_PASSWORD}@{config.LOCAL_DB_HOST}:{config.LOCAL_DB_PORT}/{config.LOCAL_DB_NAME}')

# Clear out tables for new data
response = input("This will delete all records in the database shows, restaurants, and shows_restaurants tables. Enter 'yes' to continue: ")
if response == "yes":
  with engine.connect() as connection:
    connection.execute(text("TRUNCATE TABLE shows RESTART IDENTITY CASCADE"))
    connection.execute(text("TRUNCATE TABLE restaurants RESTART IDENTITY CASCADE"))
    connection.execute(text("TRUNCATE TABLE shows_restaurants RESTART IDENTITY CASCADE"))
    connection.commit()

# Write to the database
shows.to_sql('shows', engine, if_exists='append', index=False)
food_network_restaurants.to_sql('restaurants', engine, if_exists='append', index=False)


56

We now read from the database the data we just inserted because the database has assigned them IDs and we must use those same IDs to produce the shows_restaurants join table.

In [15]:
# Read from the database to get the IDs of shows and restaurants
db_shows = pd.read_sql('SELECT * FROM shows', engine)
db_restaurants = pd.read_sql('SELECT * FROM restaurants', engine)


# Create empty dataframe with two columns for matched IDs
shows_restaurants = pd.DataFrame(columns=['show_id', 'restaurant_id'])

# Match the restaurant to the show based on their ids
for index, row in food_network.iterrows():
  show_id = db_shows[db_shows['name'] == row['show']].iloc[0]['id']
  restaurant_id = db_restaurants[(db_restaurants['name'] == row['name']) & (db_restaurants['address'] == row['address'])].iloc[0]['id']
  shows_restaurants = shows_restaurants._append({'show_id': show_id, 'restaurant_id': restaurant_id}, ignore_index=True)

# Write to join table in DB
shows_restaurants.to_sql('shows_restaurants', engine, if_exists='append', index=False)

325